<a href="https://colab.research.google.com/github/fourfeatherz/DS2002S25/blob/main/python/sql_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQLite Tutorial for SQL and SQLite

In this notebook, we will build a simple school database using SQLite. We'll:

1. Create a database schema with three tables: **Students**, **Courses**, and **Enrollments**.
2. Insert sample data: 20 students and 5 courses.
3. Enroll some students in courses such that some students are not enrolled in any course and some courses have no enrollments.
4. Run sample queries to explore the data.

Let's get started!

## Step 1: Create the Database Schema

Below is the DDL (Data Definition Language) we will use. (Note that some MySQL-specific commands have been removed or modified since SQLite does not support them.)

```sql
-- Drop tables if they exist (in reverse order to avoid FK issues)
DROP TABLE IF EXISTS Enrollments;
DROP TABLE IF EXISTS Courses;
DROP TABLE IF EXISTS Students;

-- Create Students table
CREATE TABLE Students (
    StudentID INTEGER PRIMARY KEY AUTOINCREMENT,
    FirstName VARCHAR(50) NOT NULL,
    LastName VARCHAR(50) NOT NULL,
    DateOfBirth DATE,
    Email VARCHAR(100) UNIQUE,
    EnrollmentDate DATE DEFAULT (date('now'))
);

-- Create Courses table
CREATE TABLE Courses (
    CourseID INTEGER PRIMARY KEY AUTOINCREMENT,
    CourseName VARCHAR(100) NOT NULL,
    Credits INT CHECK (Credits > 0),
    Instructor VARCHAR(100)
);

-- Create Enrollments table
CREATE TABLE Enrollments (
    EnrollmentID INTEGER PRIMARY KEY AUTOINCREMENT,
    StudentID INT,
    CourseID INT,
    EnrollmentDate DATE DEFAULT (date('now')),
    Grade CHAR(1) CHECK (Grade IN ('A', 'B', 'C', 'D', 'F'))
);
```

In [1]:
import sqlite3

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# DDL statements (using SQLite syntax)
ddl_statements = '''
DROP TABLE IF EXISTS Enrollments;
DROP TABLE IF EXISTS Courses;
DROP TABLE IF EXISTS Students;

CREATE TABLE Students (
    StudentID INTEGER PRIMARY KEY AUTOINCREMENT,
    FirstName VARCHAR(50) NOT NULL,
    LastName VARCHAR(50) NOT NULL,
    DateOfBirth DATE,
    Email VARCHAR(100) UNIQUE,
    EnrollmentDate DATE DEFAULT (date('now'))
);

CREATE TABLE Courses (
    CourseID INTEGER PRIMARY KEY AUTOINCREMENT,
    CourseName VARCHAR(100) NOT NULL,
    Credits INT CHECK (Credits > 0),
    Instructor VARCHAR(100)
);

CREATE TABLE Enrollments (
    EnrollmentID INTEGER PRIMARY KEY AUTOINCREMENT,
    StudentID INT,
    CourseID INT,
    EnrollmentDate DATE DEFAULT (date('now')),
    Grade CHAR(1) CHECK (Grade IN ('A', 'B', 'C', 'D', 'F'))
);
'''

# Execute the DDL script
cursor.executescript(ddl_statements)
conn.commit()
print('Database schema created successfully.')

Database schema created successfully.


## Step 2: Insert Data into the Students Table

We'll insert 20 students with sample names, birth dates, and emails.

In [2]:
# Sample data for 20 students (FirstName, LastName, DateOfBirth, Email)
students = [
    ('Alice', 'Smith', '2000-01-15', 'alice.smith@example.com'),
    ('Bob', 'Johnson', '1999-03-22', 'bob.johnson@example.com'),
    ('Charlie', 'Williams', '2001-07-08', 'charlie.williams@example.com'),
    ('Diana', 'Brown', '2000-11-30', 'diana.brown@example.com'),
    ('Ethan', 'Jones', '1998-05-12', 'ethan.jones@example.com'),
    ('Fiona', 'Garcia', '2002-02-20', 'fiona.garcia@example.com'),
    ('George', 'Miller', '1999-12-05', 'george.miller@example.com'),
    ('Hannah', 'Davis', '2001-09-17', 'hannah.davis@example.com'),
    ('Ian', 'Rodriguez', '2000-06-23', 'ian.rodriguez@example.com'),
    ('Julia', 'Martinez', '1998-08-14', 'julia.martinez@example.com'),
    ('Kevin', 'Hernandez', '2000-04-10', 'kevin.hernandez@example.com'),
    ('Laura', 'Lopez', '2001-01-27', 'laura.lopez@example.com'),
    ('Michael', 'Gonzalez', '1999-10-03', 'michael.gonzalez@example.com'),
    ('Nina', 'Wilson', '2002-07-19', 'nina.wilson@example.com'),
    ('Oscar', 'Anderson', '2000-03-11', 'oscar.anderson@example.com'),
    ('Paula', 'Thomas', '1998-11-25', 'paula.thomas@example.com'),
    ('Quinn', 'Taylor', '2001-05-06', 'quinn.taylor@example.com'),
    ('Rachel', 'Moore', '1999-09-09', 'rachel.moore@example.com'),
    ('Steve', 'Jackson', '2000-12-12', 'steve.jackson@example.com'),
    ('Tina', 'Martin', '2001-02-28', 'tina.martin@example.com')
]

# Insert students into the Students table
cursor.executemany('''
    INSERT INTO Students (FirstName, LastName, DateOfBirth, Email)
    VALUES (?, ?, ?, ?)
''', students)

conn.commit()
print('Inserted 20 students into the Students table.')

Inserted 20 students into the Students table.


## Step 3: Insert Data into the Courses Table

Now, let's insert 5 courses with sample names, credits, and instructors.

In [3]:
# Sample data for 5 courses (CourseName, Credits, Instructor)
courses = [
    ('Mathematics', 4, 'Dr. Euler'),
    ('Physics', 3, 'Dr. Newton'),
    ('Chemistry', 4, 'Dr. Curie'),
    ('Literature', 2, 'Prof. Shakespeare'),
    ('Art', 2, 'Prof. Da Vinci')
]

# Insert courses into the Courses table
cursor.executemany('''
    INSERT INTO Courses (CourseName, Credits, Instructor)
    VALUES (?, ?, ?)
''', courses)

conn.commit()
print('Inserted 5 courses into the Courses table.')

Inserted 5 courses into the Courses table.


## Step 4: Insert Data into the Enrollments Table

We'll enroll some students in courses. In our design:

- **Enrollments:**
  - Enroll students with IDs 1–5 in course 1,
  - Students with IDs 6–9 in course 2,
  - Students with IDs 10–11 in course 4.

This ensures that:
  - Courses 3 and 5 have no enrollments, and
  - Many students are not enrolled in any course.

In [4]:
# Sample enrollments: (StudentID, CourseID, EnrollmentDate, Grade)
enrollments = [
    (1, 1, '2023-09-01', 'A'),
    (2, 1, '2023-09-01', 'B'),
    (3, 1, '2023-09-01', 'A'),
    (4, 1, '2023-09-01', 'C'),
    (5, 1, '2023-09-01', 'B'),
    (6, 2, '2023-09-02', 'A'),
    (7, 2, '2023-09-02', 'B'),
    (8, 2, '2023-09-02', 'A'),
    (9, 2, '2023-09-02', 'C'),
    (10, 4, '2023-09-03', 'B'),
    (11, 4, '2023-09-03', 'A')
]

cursor.executemany('''
    INSERT INTO Enrollments (StudentID, CourseID, EnrollmentDate, Grade)
    VALUES (?, ?, ?, ?)
''', enrollments)

conn.commit()
print('Inserted sample enrollments into the Enrollments table.')

Inserted sample enrollments into the Enrollments table.


## Step 5: Querying the Database

Let's run some queries to explore our data.

1. **List all students, courses, and enrollments.**
2. **Find students not enrolled in any course.**
3. **Find courses with no enrollments.**

In [5]:
# Query 1: List all students
print('All Students:')
for row in cursor.execute('SELECT * FROM Students'):
    print(row)

print('\nAll Courses:')
for row in cursor.execute('SELECT * FROM Courses'):
    print(row)

print('\nAll Enrollments:')
for row in cursor.execute('SELECT * FROM Enrollments'):
    print(row)

# Query 2: List students not enrolled in any course
print('\nStudents not enrolled in any course:')
cursor.execute('''
    SELECT * FROM Students
    WHERE StudentID NOT IN (SELECT StudentID FROM Enrollments)
''')
for row in cursor.fetchall():
    print(row)

# Query 3: List courses with no enrollments
print('\nCourses with no enrollments:')
cursor.execute('''
    SELECT * FROM Courses
    WHERE CourseID NOT IN (SELECT CourseID FROM Enrollments)
''')
for row in cursor.fetchall():
    print(row)


All Students:
(1, 'Alice', 'Smith', '2000-01-15', 'alice.smith@example.com', '2025-02-03')
(2, 'Bob', 'Johnson', '1999-03-22', 'bob.johnson@example.com', '2025-02-03')
(3, 'Charlie', 'Williams', '2001-07-08', 'charlie.williams@example.com', '2025-02-03')
(4, 'Diana', 'Brown', '2000-11-30', 'diana.brown@example.com', '2025-02-03')
(5, 'Ethan', 'Jones', '1998-05-12', 'ethan.jones@example.com', '2025-02-03')
(6, 'Fiona', 'Garcia', '2002-02-20', 'fiona.garcia@example.com', '2025-02-03')
(7, 'George', 'Miller', '1999-12-05', 'george.miller@example.com', '2025-02-03')
(8, 'Hannah', 'Davis', '2001-09-17', 'hannah.davis@example.com', '2025-02-03')
(9, 'Ian', 'Rodriguez', '2000-06-23', 'ian.rodriguez@example.com', '2025-02-03')
(10, 'Julia', 'Martinez', '1998-08-14', 'julia.martinez@example.com', '2025-02-03')
(11, 'Kevin', 'Hernandez', '2000-04-10', 'kevin.hernandez@example.com', '2025-02-03')
(12, 'Laura', 'Lopez', '2001-01-27', 'laura.lopez@example.com', '2025-02-03')
(13, 'Michael', 'Gonzal

## Conclusion

In this notebook, we created a school database using SQLite, inserted sample data, and executed queries to explore the information. This example is a great starting point for learning SQL and SQLite in an interactive environment like Google Colab.

In [6]:
# Close the database connection
conn.close()
print('Database connection closed.')

Database connection closed.
